In [1]:
import os
import pandas as pd
import geopandas as gpd
import scipy
import numpy as np
from tqdm import tqdm
from shapely.geometry import Point
from sklearn.impute import KNNImputer

In [2]:
start_date = '2020-03-01'
end_date = '2022-03-31'
long_term_exposure_years = 5
N_neighbors_spatial_interpolation = 3
N_neighbors_temporal_interpolation = 3

# Met data

In [3]:
meo_stations = {}
for idx, line in enumerate(open(os.path.join("meteorology", "SRCE.DATA")).readlines()):
    if idx % 4 == 0:
        tokens = line.split(",")
        src_id = int(tokens[0])
        lat = tokens[-3].strip()
        lon = tokens[-2].strip()
        if lat != "" and lon != "":
            meo_stations[src_id] = [float(lon), float(lat)]

In [4]:
# https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/ukmo_guide.html
# https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/WH_Table.html
# https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/WD_Table.html
# https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/RH_Table.html
# https://artefacts.ceda.ac.uk/badc_datadocs/ukmo-midas/RD_Table.html

with open(os.path.join("meteorology", "WH_Column_Headers.txt")) as f:
    meo_data_header = f.readlines()
    meo_data_header = list(map(lambda s: s.strip(), meo_data_header[0].strip().split(",")))

meo_2020 = pd.read_csv(os.path.join("meteorology", "midas_wxhrly_202001-202012.txt"), na_values=["", " "], names=meo_data_header, header=None, low_memory=False)
meo_2021 = pd.read_csv(os.path.join("meteorology", "midas_wxhrly_202101-202112.txt"), na_values=["", " "], names=meo_data_header, header=None, low_memory=False)
meo_2022 = pd.read_csv(os.path.join("meteorology", "midas_wxhrly_202201-202212.txt"), na_values=["", " "], names=meo_data_header, header=None, low_memory=False)

meo_df = pd.concat([meo_2020, meo_2021, meo_2022], axis=0).reset_index()

meo_df['OB_TIME'] = pd.to_datetime(meo_df['OB_TIME'])
meo_df = meo_df[(meo_df["OB_TIME"] >= start_date) & (meo_df["OB_TIME"] <= end_date)]
meo_df['OB_TIME'] = meo_df['OB_TIME'].dt.strftime('%Y-%m-%d')
weather_variables = ['AIR_TEMPERATURE', 'RLTV_HUM', 'MSL_PRESSURE',
                     'WIND_SPEED', 'VISIBILITY', 'WMO_HR_SUN_DUR', 'CLD_TTL_AMT_ID']
meo_df = meo_df[['OB_TIME', 'SRC_ID'] + weather_variables]
# average hourly data into daily means
meo_df = meo_df.groupby(['OB_TIME', 'SRC_ID'])[weather_variables].mean().reset_index()
meo_df['SRC_ID'] = meo_df['SRC_ID'].astype(int)

In [ ]:
# add rainfall data

with open(os.path.join("meteorology", "RD_Column_Headers.csv")) as f:
    rainfall_data_header = f.readlines()
    rainfall_data_header = list(
        map(lambda s: s.strip(), rainfall_data_header[0].strip().split(",")))

rainfall_2020 = pd.read_csv(os.path.join("meteorology", "midas_raindrnl_202001-202012.txt"), na_values=["", " "], names=rainfall_data_header, header=None, low_memory=False)
rainfall_2021 = pd.read_csv(os.path.join("meteorology", "midas_raindrnl_202101-202112.txt"), na_values=["", " "], names=rainfall_data_header, header=None, low_memory=False)
rainfall_2022 = pd.read_csv(os.path.join("meteorology", "midas_raindrnl_202201-202212.txt"), na_values=["", " "], names=rainfall_data_header, header=None, low_memory=False)

rainfall_df = pd.concat([rainfall_2020, rainfall_2021, rainfall_2022], axis=0).reset_index()
rainfall_df['OB_DATE'] = pd.to_datetime(rainfall_df['OB_DATE'])
rainfall_df = rainfall_df[(rainfall_df["OB_DATE"] >= start_date) & (rainfall_df["OB_DATE"] <= end_date)]
rainfall_df['OB_TIME'] = rainfall_df['OB_DATE'].dt.strftime('%Y-%m-%d')
rainfall_df = rainfall_df[['OB_TIME', 'SRC_ID', 'PRCP_AMT']]
rainfall_df.head()

meo_df = pd.merge(left=meo_df, right=rainfall_df, left_on=['OB_TIME', 'SRC_ID'], right_on=['OB_TIME', 'SRC_ID'], how="left")
weather_variables = weather_variables + ['PRCP_AMT']
meo_df.head()

In [ ]:
meo_df['coords'] = meo_df['SRC_ID'].map(meo_stations)
meo_df = meo_df[~meo_df['coords'].isnull()]
meo_df['lon'] = meo_df['coords'].apply(lambda x: x[0])
meo_df['lat'] = meo_df['coords'].apply(lambda x: x[1])
meo_df['coords'] = meo_df['coords'].apply(Point)
meo_df.head()

In [ ]:
ltlas_gdf = gpd.GeoDataFrame.from_file(os.path.join("gis", 'lad19.geojson'))
ltlas_gdf.head()

In [ ]:
meo_gdf = gpd.GeoDataFrame(meo_df, geometry='coords', crs=4326)  # WGS84
meo_gdf = meo_gdf.to_crs(ltlas_gdf.crs)
meo_gdf.head()

In [9]:
# spatial join
pointInPolys = gpd.tools.sjoin(meo_gdf, ltlas_gdf, predicate="within", how='right')

# remove non-UK stations
non_UK_stations = pointInPolys[pointInPolys['district_id'].isnull()]['SRC_ID'].unique()
meo_gdf = meo_gdf[~meo_gdf['SRC_ID'].isin(non_UK_stations)]
pointInPolys = pointInPolys[~pointInPolys['SRC_ID'].isin(non_UK_stations)]

In [ ]:
meo_ltla_gdf = pointInPolys[['OB_TIME', 'SRC_ID', 'district_id',
                             'district_name', 'lat', 'lon', 'district_lat', 'district_lon'] + weather_variables]
meo_ltla_gdf = meo_ltla_gdf.rename(columns={'OB_TIME': 'date',
                                            'AIR_TEMPERATURE': 'temperature',
                                            'RLTV_HUM': 'relative_humidity',
                                            'MSL_PRESSURE': 'pressure',
                                            'WIND_SPEED': 'wind_speed',
                                            'VISIBILITY': 'visibility',
                                            'WMO_HR_SUN_DUR': 'sunshine',
                                            'CLD_TTL_AMT_ID': 'cloud_amount',
                                            'PRCP_AMT': 'precipitation',
                                            'SRC_ID': 'station_id',
                                            'lat': 'station_lat', 'lon': 'station_lon'})
new_weather_variables = ['temperature', 'relative_humidity',
                         'pressure', 'wind_speed', 'visibility', 'sunshine', 'cloud_amount', 'precipitation']
meo_ltla_gdf['date'] = pd.to_datetime(meo_ltla_gdf['date'])
stations = meo_ltla_gdf['station_id'].unique()
districts = meo_ltla_gdf['district_name'].unique()
meo_ltla_gdf.head()

In [ ]:
print("# of meteorology stations: %s" % len(stations))
print("# of districts: %s" % len(ltlas_gdf['district_name'].unique()))

In [12]:
# for districts without stations, fill empty values
districts_without_stations = meo_ltla_gdf[meo_ltla_gdf['station_id'].isnull()]
dates = pd.date_range(start=start_date, end=end_date)
new_data = []
for idx, row in districts_without_stations.iterrows():
    for date in dates:
        record = {
            "date": date,
            "station_id": row.station_id,
            "district_id": row.district_id,
            "district_name": row.district_name,
            "station_lat": row.station_lat,
            "station_lon": row.station_lon,
            "district_lat": row.district_lat,
            "district_lon": row.district_lon,
        }
        for weather_variable in new_weather_variables:
            record[weather_variable] = row[weather_variable]
        new_data.append(record)
districts_without_stations_filled_dates = pd.DataFrame.from_records(new_data)
new_meo_ltla_gdf = pd.concat([meo_ltla_gdf[~meo_ltla_gdf['station_id'].isnull()], districts_without_stations_filled_dates], axis=0)
new_meo_ltla_gdf = new_meo_ltla_gdf.sort_values(['district_name', 'station_id', 'date'])


In [13]:
new_data = []
for station_id in stations:
    current_station_dates = new_meo_ltla_gdf[new_meo_ltla_gdf['station_id'] == station_id]['date'].unique()
    if len(dates) != len(current_station_dates):
        districts = new_meo_ltla_gdf[new_meo_ltla_gdf['station_id'] == station_id]['district_name'].unique()
        for district_name in districts:
            one_row = new_meo_ltla_gdf[(new_meo_ltla_gdf['station_id'] == station_id) & (
                new_meo_ltla_gdf['district_name'] == district_name)].head(1)
            for date in dates:
                if date not in current_station_dates:
                    record = {
                        "date": date,
                        "station_id": station_id,
                        "district_id": one_row.district_id.values[0],
                        "district_name": district_name,
                        "station_lat": one_row.station_lat.values[0],
                        "station_lon": one_row.station_lon.values[0],
                        "district_lat": one_row.district_lat.values[0],
                        "district_lon": one_row.district_lon.values[0],
                    }
                    for weather_variable in new_weather_variables:
                        record[weather_variable] = np.nan
                    new_data.append(record)
missing_records_df = pd.DataFrame.from_records(new_data)
new_meo_ltla_gdf = pd.concat([new_meo_ltla_gdf, missing_records_df], axis=0)
new_meo_ltla_gdf = new_meo_ltla_gdf.sort_values(['district_name', 'station_id', 'date'])

In [ ]:
# make sure no date is missing for each district
for district in districts:
    assert(len(dates) == len(new_meo_ltla_gdf[new_meo_ltla_gdf['district_name'] == district]['date'].unique()))

# make sure no district is missing for each date
for date in dates:
    one_day_df = new_meo_ltla_gdf[new_meo_ltla_gdf['date'] == date]
    assert(len(one_day_df['district_name'].unique()) == len(ltlas_gdf['district_name'].unique()))

new_meo_ltla_gdf = new_meo_ltla_gdf.reset_index()
new_meo_ltla_gdf = new_meo_ltla_gdf.drop('index', axis=1)
new_meo_ltla_gdf.head()


In [ ]:
# for districts that have no meteorology monitoring station,
# use the district's location as the station location for spatial interpolation
new_meo_ltla_gdf['station_lat'] = new_meo_ltla_gdf['station_lat'].fillna(value=new_meo_ltla_gdf['district_lat'])
new_meo_ltla_gdf['station_lon'] = new_meo_ltla_gdf['station_lon'].fillna(value=new_meo_ltla_gdf['district_lon'])
new_meo_ltla_gdf = new_meo_ltla_gdf.sort_values(by=['district_name', 'station_id', 'date'])
new_meo_ltla_gdf.head()


In [ ]:
# Average daily values into LTLAs
new_meo_ltla_gdf['district_name'] = new_meo_ltla_gdf['district_name'].astype(str)
new_meo_ltla_gdf['district_id'] = new_meo_ltla_gdf['district_id'].astype(str)
agg_function = {}
for weather_variable in new_weather_variables:
    agg_function[weather_variable] = "mean"
new_meo_ltla_gdf = new_meo_ltla_gdf.groupby(by=['date', 'district_name', 'district_id', 'district_lon', 'district_lat']).agg("mean").reset_index()
new_meo_ltla_gdf.head()

In [ ]:
# Temporal interpolation
for district in new_meo_ltla_gdf['district_name'].unique():
    for weather_variable in new_weather_variables:
        district_df = new_meo_ltla_gdf[new_meo_ltla_gdf['district_name'] == district].reset_index().copy()
        weather_values = district_df[weather_variable].interpolate(method="linear", limit=N_neighbors_temporal_interpolation)
        new_meo_ltla_gdf.loc[(new_meo_ltla_gdf['district_name'] == district), weather_variable] = new_meo_ltla_gdf.loc[(new_meo_ltla_gdf['district_name'] == district), weather_variable].fillna(value=weather_values)
new_meo_ltla_gdf.head()

In [ ]:
# Spatial interpolation
def spatial_distance(X1, X2, missing_values=None):
    # X: ..., 'district_lon', 'district_lat'
    return scipy.spatial.distance.euclidean(X1[-2:], X2[-2:])


for date in tqdm(dates, desc="Meteorology data spatial interpolation"):
    weather_matrix = new_meo_ltla_gdf[new_meo_ltla_gdf['date'] == date][new_weather_variables + ['district_lon', 'district_lat']]
    imputer = KNNImputer(n_neighbors=N_neighbors_spatial_interpolation, weights='distance', metric=spatial_distance)
    imputed_matrix = imputer.fit_transform(weather_matrix)
    imputed_df = pd.DataFrame(data=imputed_matrix[:, 0:len(new_weather_variables)], index=weather_matrix.index, columns=new_weather_variables)
    new_meo_ltla_gdf.loc[(new_meo_ltla_gdf['date'] == date), new_weather_variables] = new_meo_ltla_gdf.loc[(
        new_meo_ltla_gdf['date'] == date), new_weather_variables].fillna(value=imputed_df)

In [19]:
new_meo_ltla_gdf = new_meo_ltla_gdf.drop('district_lon', axis=1)
new_meo_ltla_gdf = new_meo_ltla_gdf.drop('district_lat', axis=1)

new_meo_ltla_gdf[new_weather_variables] = new_meo_ltla_gdf[new_weather_variables].astype(float)
new_meo_ltla_gdf['date'] = new_meo_ltla_gdf['date'].dt.strftime('%Y-%m-%d')


# UV data

In [20]:
station_ids = ['bel', 'cam', 'chi', 'gla', 'inv',
               'lee', 'ler', 'mal', 'wey', 'mcr', 'swa', 'rdg']
station_names = ['Belfast', 'Camborne', 'Chilton', 'Glasgow', 'Inverness', 'Leeds',
                 'Lerwick', 'Malin Head', 'Weybourne', 'Manchester', "Swansea", "Reading"]
# https://uk-air.defra.gov.uk/research/ozone-uv/ozone-monitoring-stations
# https://uk-air.defra.gov.uk/networks/site-info?site_id=WEYB
station_locations = ['54.595940, -5.826939', '50.218485, -5.327063', '51.575002, -1.317720', '55.862468, -4.344691', '57.473432, -4.193860', '53.845083, -1.614636',
                     '60.139220, -1.185319', '55.371731, -7.339425', '52.950490, 1.122017', '53.474417, -2.233773', '51.609444, -3.984910', '51.441509, -0.937416']
assert(len(station_ids) == len(station_locations))
station_location_map = {}
for station_id, station_location in zip(station_ids, station_locations):
    station_location_map[station_id] = station_location

In [ ]:
data = []
for station_id in station_location_map:
    for date in pd.date_range(start=start_date, end=end_date):
        record = {
            "station_id": station_id,
            "station_lon": float(station_location_map[station_id].split(",")[1]),
            "station_lat": float(station_location_map[station_id].split(",")[0]),
            "date": date,
        }
        date_str = date.strftime("%d-%b-%y")
        if os.path.exists("meteorology/uv_index/results/%s/%s.txt" % (station_id, date_str)):
            with open("meteorology/uv_index/results/%s/%s.txt" % (station_id, date_str)) as f:
                text = f.read()
                if "404 Not Found" in text:
                    value = np.nan
                else:
                    tokens = text.split("=")[1].strip().split("@")[:-1]
                    values = []
                    for i in range(0, len(tokens)-1, 2):
                        values.append(float(tokens[i+1]))
                    value = np.nanmean(values)
                    value = 0.06081034*value+0.18294901626674043
        else:
            value = np.nan
        record['uv_index'] = value
        data.append(record)
uv_df = pd.DataFrame.from_records(data)
uv_df.head()

In [ ]:
ltlas_gdf = gpd.GeoDataFrame.from_file(os.path.join("gis", 'lad19.geojson'))
ltlas_gdf.head()

In [ ]:
uv_df['geometry'] = uv_df[['station_lon', 'station_lat']].apply(lambda x: [x.iloc[0], x.iloc[1]], axis=1)
uv_df['geometry'] = uv_df['geometry'].apply(Point)
uv_df = gpd.GeoDataFrame(uv_df, geometry='geometry', crs=4326)  # WGS84
uv_df = uv_df.to_crs(ltlas_gdf.crs)
uv_df.head()

In [ ]:
# spatial join
pointInPolys = gpd.tools.sjoin(uv_df, ltlas_gdf, predicate="within", how='right')

# remove non-UK stations
non_UK_stations = pointInPolys[pointInPolys['district_id'].isnull()]['station_id'].unique()
meo_gdf = uv_df[~uv_df['station_id'].isin(non_UK_stations)]
pointInPolys = pointInPolys[~pointInPolys['station_id'].isin(non_UK_stations)]

uv_ltla_gdf = pointInPolys[['date', 'station_id', 'district_id',
                            'district_name', 'station_lat', 'station_lon', 'district_lat', 'district_lon'] + ['uv_index']]
uv_ltla_gdf.head()

In [ ]:
districts_without_stations = uv_ltla_gdf[uv_ltla_gdf['station_id'].isnull()]
dates = pd.date_range(start=start_date, end=end_date)
new_data = []
for idx, row in districts_without_stations.iterrows():
    for date in dates:
        record = {
            "date": date,
            "station_id": row.station_id,
            "district_id": row.district_id,
            "district_name": row.district_name,
            "station_lat": row.station_lat,
            "station_lon": row.station_lon,
            "district_lat": row.district_lat,
            "district_lon": row.district_lon,
            'uv_index': row.uv_index
        }
        new_data.append(record)
districts_without_stations_filled_dates = pd.DataFrame.from_records(new_data)
uv_ltla_gdf = pd.concat([uv_ltla_gdf[~uv_ltla_gdf['station_id'].isnull()], districts_without_stations_filled_dates], axis=0)
uv_ltla_gdf = uv_ltla_gdf.sort_values(['district_name', 'station_id', 'date'])
uv_ltla_gdf.head()

In [ ]:
new_data = []
for station_id in station_ids:
    current_station_dates = uv_ltla_gdf[uv_ltla_gdf['station_id'] == station_id]['date'].unique()
    if len(dates) != len(current_station_dates):
        districts = uv_ltla_gdf[uv_ltla_gdf['station_id'] == station_id]['district_name'].unique()
        for district_name in districts:
            one_row = uv_ltla_gdf[(uv_ltla_gdf['station_id'] == station_id) & (uv_ltla_gdf['district_name'] == district_name)].head(1)
            for date in dates:
                if date not in current_station_dates:
                    record = {
                        "date": date,
                        "station_id": station_id,
                        "district_id": one_row.district_id.values[0],
                        "district_name": district_name,
                        "station_lat": one_row.station_lat.values[0],
                        "station_lon": one_row.station_lon.values[0],
                        "district_lat": one_row.district_lat.values[0],
                        "district_lon": one_row.district_lon.values[0],
                        'uv_index': np.nan
                    }
                    new_data.append(record)
missing_records_df = pd.DataFrame.from_records(new_data)
uv_ltla_gdf = pd.concat([uv_ltla_gdf, missing_records_df], axis=0)
uv_ltla_gdf = uv_ltla_gdf.sort_values(['district_name', 'station_id', 'date'])
uv_ltla_gdf.head()

In [ ]:
# make sure no date is missing for each district
for district in districts:
    assert(len(dates) == len(uv_ltla_gdf[uv_ltla_gdf['district_name'] == district]['date'].unique()))

# make sure no district is missing for each date
for date in dates:
    one_day_df = uv_ltla_gdf[uv_ltla_gdf['date'] == date]
    assert(len(one_day_df['district_name'].unique())== len(ltlas_gdf['district_name'].unique()))

uv_ltla_gdf = uv_ltla_gdf.reset_index()
uv_ltla_gdf = uv_ltla_gdf.drop('index', axis=1)
uv_ltla_gdf.head()

In [ ]:
# for districts that have no UV monitoring station,
# use the district's location as the station location for spatial interpolation
uv_ltla_gdf['station_lat'] = uv_ltla_gdf['station_lat'].fillna(value=uv_ltla_gdf['district_lat'])
uv_ltla_gdf['station_lon'] = uv_ltla_gdf['station_lon'].fillna(value=uv_ltla_gdf['district_lon'])
uv_ltla_gdf = uv_ltla_gdf.sort_values(by=['district_name', 'station_id', 'date'])
uv_ltla_gdf.head()

In [ ]:
# Average daily values into LTLAs
uv_ltla_gdf['district_name'] = uv_ltla_gdf['district_name'].astype(str)
uv_ltla_gdf['district_id'] = uv_ltla_gdf['district_id'].astype(str)
agg_function = {}
agg_function['uv_index'] = "mean"
uv_ltla_gdf = uv_ltla_gdf.groupby(by=['date', 'district_name', 'district_id', 'district_lon', 'district_lat']).agg(agg_function).reset_index()
uv_ltla_gdf.head()

In [ ]:
# Temporal interpolation
for district in uv_ltla_gdf['district_name'].unique():
    for weather_variable in ['uv_index']:
        district_df = uv_ltla_gdf[uv_ltla_gdf['district_name'] == district].reset_index(
        ).copy()
        weather_values = district_df[weather_variable].interpolate(method="linear", limit=N_neighbors_temporal_interpolation)
        uv_ltla_gdf.loc[(uv_ltla_gdf['district_name'] == district), weather_variable] = uv_ltla_gdf.loc[(uv_ltla_gdf['district_name'] == district), weather_variable].fillna(value=weather_values)
uv_ltla_gdf.head()

In [ ]:
# Spatial interpolation
def spatial_distance(X1, X2, missing_values=None):
    # X: ..., 'district_lon', 'district_lat'
    return scipy.spatial.distance.euclidean(X1[-2:], X2[-2:])


for date in tqdm(dates, desc="UV data spatial interpolation"):
    weather_matrix = uv_ltla_gdf[uv_ltla_gdf['date'] == date][['uv_index'] + ['district_lon', 'district_lat']]
    imputer = KNNImputer(n_neighbors=N_neighbors_spatial_interpolation, weights='distance', metric=spatial_distance)
    imputed_matrix = imputer.fit_transform(weather_matrix)
    imputed_df = pd.DataFrame(data=imputed_matrix[:, 0:len(['uv_index'])], index=weather_matrix.index, columns=['uv_index'])
    uv_ltla_gdf.loc[(uv_ltla_gdf['date'] == date), ['uv_index']] = uv_ltla_gdf.loc[(uv_ltla_gdf['date'] == date), ['uv_index']].fillna(value=imputed_df)

In [32]:
uv_ltla_gdf = uv_ltla_gdf.drop('district_lon', axis=1)
uv_ltla_gdf = uv_ltla_gdf.drop('district_lat', axis=1)

uv_ltla_gdf[['uv_index']] = uv_ltla_gdf[['uv_index']].astype(float)
uv_ltla_gdf['date'] = uv_ltla_gdf['date'].dt.strftime('%Y-%m-%d')

In [33]:
meteorology_ltla_gdf = pd.merge(new_meo_ltla_gdf, uv_ltla_gdf, left_on=("date", 'district_name', 'district_id'), right_on=("date", 'district_name', 'district_id'))
meteorology_ltla_gdf.to_csv("../meteorology_with_uv.csv", float_format="%.1f", na_rep="N/A", index=False)